In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
df = pd.read_csv(r"C:\Users\Mritunjoy Paul\CSV files\Water_Quality_Dataset.csv")

In [3]:
df.head()

,Timestamp,Location,pH,Turbidity (NTU),Temperature (°C),DO (mg/L),BOD (mg/L),Lead (mg/L),Mercury (mg/L),Arsenic (mg/L),Pollution_Level
0,2024-01-01 00:00:00,L4,8.762414,14.468306,16.461631,6.258335,2.478582,0.002801,0.001924,0.018769,2
1,2024-01-01 01:00:00,L5,7.477966,1.905645,27.446865,2.826120,8.048453,0.019551,0.000633,0.003708,2
2,2024-01-01 02:00:00,L3,7.938278,1.889506,34.623557,3.380191,2.102481,0.019291,0.000205,0.005055,2
3,2024-01-01 03:00:00,L5,8.728748,0.736115,18.802153,6.543794,6.714160,0.003358,0.001058,0.005638,2
4,2024-01-01 04:00:00,L5,7.975335,19.151777,30.851902,4.370861,2.114614,0.006059,0.000124,0.001237,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Timestamp         1000 non-null   object 
 1   Location          1000 non-null   object 
 2   pH                1000 non-null   float64
 3   Turbidity (NTU)   1000 non-null   float64
 4   Temperature (°C)  1000 non-null   float64
 5   DO (mg/L)         1000 non-null   float64
 6   BOD (mg/L)        1000 non-null   float64
 7   Lead (mg/L)       1000 non-null   float64
 8   Mercury (mg/L)    1000 non-null   float64
 9   Arsenic (mg/L)    1000 non-null   float64
 10  Pollution_Level   1000 non-null   int64  
dtypes: float64(8), int64(1), object(2)
memory usage: 86.1+ KB


In [5]:
df.describe()

,pH,Turbidity (NTU),Temperature (°C),DO (mg/L),BOD (mg/L),Lead (mg/L),Mercury (mg/L),Arsenic (mg/L),Pollution_Level
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,7.250855,10.218681,24.966871,5.928647,5.482985,0.009965,0.000981,0.009855,1.909000
std,1.024515,5.631584,5.756625,2.287483,2.604094,0.005746,0.000569,0.005544,0.307919
min,5.516212,0.502627,15.000233,2.000246,1.008491,0.000128,0.000010,0.000505,0.000000
25%,6.324944,5.566377,19.950098,3.984953,3.242944,0.005000,0.000489,0.005015,2.000000
50%,7.265239,10.236904,25.041492,5.871292,5.368891,0.009894,0.000977,0.009392,2.000000
75%,8.107655,15.148932,29.967260,7.841829,7.730134,0.015130,0.001444,0.014484,2.000000
max,8.997948,19.967776,34.991154,9.981995,9.994153,0.019989,0.001998,0.019922,2.000000


In [6]:
print("\nMissing Values:\n", df.isnull().sum())


Missing Values:
 Timestamp           0
Location            0
pH                  0
Turbidity (NTU)     0
Temperature (°C)    0
DO (mg/L)           0
BOD (mg/L)          0
Lead (mg/L)         0
Mercury (mg/L)      0
Arsenic (mg/L)      0
Pollution_Level     0
dtype: int64


In [7]:
threshold = 0.4 * len(df)
df = df.dropna(thresh=threshold, axis=1)

In [8]:
for col in df.columns:
    if df[col].dtype == "object":  
        df[col] = df[col].fillna(df[col].mode()[0])   # fill categorical with mode
    else:
        df[col] = df[col].fillna(df[col].median())   # fill numerical with median

In [9]:
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.sort_values("date")

In [10]:
num_cols = df.select_dtypes(include=np.number).columns

In [11]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower, lower, df[col])
    df[col] = np.where(df[col] > upper, upper, df[col])

In [12]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [13]:
print("\nCleaned Dataset:\n", df.head())


Cleaned Dataset:
              Timestamp Location        pH  Turbidity (NTU)  Temperature (°C)  \
0  2024-01-01 00:00:00       L4  1.476128         0.754983         -1.478209   
1  2024-01-01 01:00:00       L5  0.221788        -1.476884          0.431022   
2  2024-01-01 02:00:00       L3  0.671310        -1.479751          1.678330   
3  2024-01-01 03:00:00       L5  1.443251        -1.684661         -1.071427   
4  2024-01-01 04:00:00       L5  0.707499         1.587043          1.022817   

   DO (mg/L)  BOD (mg/L)  Lead (mg/L)  Mercury (mg/L)  Arsenic (mg/L)  \
0   0.144199   -1.154300    -1.247342        1.657955        1.608575   
1  -1.356985    0.985660     1.668932       -0.612683       -1.109246   
2  -1.114645   -1.298799     1.623595       -1.364468       -0.866242   
3   0.269053    0.473021    -1.150387        0.136206       -0.761048   
4  -0.681345   -1.294138    -0.680237       -1.507774       -1.555122   

   Pollution_Level  
0              0.0  
1              0.0 

In [14]:
# Save preprocessed data
df.to_csv("preprocessed_water_quality.csv", index=False)